<a href="https://colab.research.google.com/github/abdinegara783/FindIT2023/blob/Experiment/experiment_dian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

# Import Library/Package

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Import Dataset

In [8]:
#Path from Dataset
TrainingFeatures_path='/content/drive/MyDrive/FindIT/training_set_features.csv'
TrainingLabels_path='/content/drive/MyDrive/FindIT/training_set_labels.csv'

##Preprocessing TrainingFeatue_path

In [9]:
# Membaca file CSV sebagai DataFrame
#Reading the TrainingFeatures CSV file as DataFrame
df_TrainingFeatures = pd.read_csv(TrainingFeatures_path)


# shows 5 first row from DataFrame
df_TrainingFeatures.head()

FileNotFoundError: ignored

In [ ]:
#shows how many feature are there
print('This is the feature from datasets:\n{}'.format(df_TrainingFeatures.columns.tolist()))
print('\nThis datasets has {} Feature'.format(len(df_TrainingFeatures.columns)))

In [ ]:
# Memeriksa tipe data dari tiap kolom
print(df_TrainingFeatures.dtypes)

In [ ]:
# melihat kolom bertipe data object
kolom = ['age_group', 'education', 'race', 'sex', 'income_poverty', 'marital_status', 'rent_or_own', 'employment_status', 'hhs_geo_region', 'census_msa', 'employment_industry', 'employment_occupation']
df_TrainingFeatures[kolom].head()


In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
for kolom in df_TrainingFeatures[kolom].columns:
    df_TrainingFeatures[kolom] = label_encoder.fit_transform(df_TrainingFeatures[kolom])


In [ ]:
# melihat kolom bertipe data object
kolom = ['age_group', 'education', 'race', 'sex', 'income_poverty', 'marital_status', 'rent_or_own', 'employment_status', 'hhs_geo_region', 'census_msa', 'employment_industry', 'employment_occupation']
df_TrainingFeatures[kolom].head()

In [ ]:
# Mengecek apakah ada nilai yang hilang pada setiap kolom
missing_data = df_TrainingFeatures.isna().any(axis=0)
missing_data= missing_data[missing_data == True].index.tolist()
# Mencetak nama kolom yang memiliki nilai hilang
print("Kolom dengan nilai hilang: {}".format(missing_data))


In [ ]:
# Melihat baris yang hilang dari kolom tertentu
def find_missing_rows(df, column_name):
    # Mencari baris yang memiliki nilai hilang pada kolom tertentu
    missing_data = df[column_name].isna()
    missing_rows = df[missing_data]
    missing_indices = missing_rows.index.tolist()
    return missing_indices
find_missing_rows(df_TrainingFeatures,missing_data[0])

In [ ]:
# Memeriksa tipe data dari variabel missing_data
print(df_TrainingFeatures.dtypes)

In [ ]:
# Memilih kolom-kolom DataFrame yang memiliki tipe data numerik
df_numerik = df_TrainingFeatures.select_dtypes(include=['int64', 'float64'])

# Memeriksa DataFrame hasil pemilihan
print(df_numerik.head())

In [ ]:
# Melihat jumlah nilai unik dalam setiap kolom
print(df_numerik.nunique())

In [ ]:
# create correlation matrix
corr = df_TrainingFeatures.corr()
# Membuat heatmap
fig, ax = plt.subplots(figsize=(15, 15)) # menambahkan figsize
sns.heatmap(corr, cmap='coolwarm', annot=True, ax=ax)



In [7]:
# Cari nilai modus dari setiap kolom
mode_values = df_TrainingFeatures.mode()
df_TrainingFeatures.fillna(mode_values, inplace=True)

NameError: ignored

In [ ]:
mode_values

In [ ]:
df_TrainingFeatures.isnull()
df_TrainingFeatures[df_TrainingFeatures == True].index.tolist()

# EDA (Explanatory Data Analysis)

In [ ]:

def visualize_data(df):
    """
    Fungsi untuk menampilkan visualisasi banyaknya variabel pada setiap kolom dataframe.
    """
    # Cek tipe data setiap kolom
    column_types = df.dtypes

    # Looping untuk setiap kolom
    for column in df.columns:
        # Hanya tampilkan visualisasi jika tipe datanya numerik
        if column_types[column] in ['int64', 'float64']:
            # Buat histogram
            df[column].hist(bins=10)
            plt.title(f'Histogram of {column}')
            plt.show()
visualize_data(df_TrainingFeatures)

In [ ]:
print(df_TrainingFeatures.isnull().sum())

In [ ]:
df_TrainingFeatures.copy

In [ ]:
df_TrainingFeatures = df_TrainingFeatures.fillna(df_TrainingFeatures.mode().iloc[0])

# Cek missing value setelah diisi dengan modus
print(df_TrainingFeatures.isnull().sum())

In [ ]:
# Melihat semua kolom pada dataframe yang bukan numerik
non_numeric_columns = df_TrainingFeatures.select_dtypes(exclude=['int64', 'float64']).columns

print('Kolom yang bukan numerik:')
df_TrainingFeatures[non_numeric_columns].nunique()


In [ ]:
df_TrainingFeatures['employment_occupation'].dtype

In [ ]:
df_TrainingLabels = pd.read_csv(TrainingLabels_path)
df_TrainingLabels

# Feature Enginering

# Feature Selection

In [ ]:
import numpy as np
import random
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

def objective_function(X, y, feature_subset):
    model = LogisticRegression()
    scores = cross_val_score(model, X[:, feature_subset], y, cv=5)
    return np.mean(scores)

def binary_bat_algorithm(X, y, n_bats=10, n_iterations=100, a=0.9, r0=0.5):
    n_features = X.shape[1]
    bats = np.random.randint(0, 2, (n_bats, n_features))
    velocities = np.random.uniform(-1, 1, (n_bats, n_features))
    frequencies = np.random.uniform(0, 2, n_bats)
    best_bat = bats[np.argmax([objective_function(X, y, bat) for bat in bats])]

    for _ in range(n_iterations):
        for i, bat in enumerate(bats):
            velocities[i] += (bat - best_bat) * frequencies[i]
            new_bat = (bat + velocities[i]).clip(0, 1)
            new_bat = (np.random.rand(n_features) < new_bat).astype(int)

            if objective_function(X, y, new_bat) > objective_function(X, y, bat):
                bats[i] = new_bat

            if np.random.rand() > r0:
                new_bat = best_bat + np.random.uniform(-a, a, n_features)
                new_bat = (np.random.rand(n_features) < new_bat).astype(int)

                if objective_function(X, y, new_bat) > objective_function(X, y, bat):
                    bats[i] = new_bat

        best_bat = bats[np.argmax([objective_function(X, y, bat) for bat in bats])]

    return best_bat



In [ ]:
df_TrainingLabels['h1n1_vaccine']

In [ ]:
X, y = df_TrainingFeatures.values, df_TrainingLabels['h1n1_vaccine'].values
selected_features = binary_bat_algorithm(X, y)
print("Selected features:", selected_features)

In [ ]:
unique_classes = df_TrainingLabels['h1n1_vaccine'].unique()
num_classes = len(unique_classes)
print(num_classes)

# Modeling

xgboost standard

In [ ]:
 pip install xgboost

In [ ]:
X.head()

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2)


In [ ]:
D_train = xgb.DMatrix(X_train, label=Y_train)
D_test = xgb.DMatrix(X_test, label=Y_test)

In [ ]:
param = {
    'eta': 0.3, 
    'max_depth': 3,  
    'objective': 'multi:softprob',  
    'num_class': 2} 

steps = 99  # The number of training iterations

In [ ]:
model = xgb.train(param, D_train, steps)

In [ ]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score

preds = model.predict(X_test)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Precision = {}".format(precision_score(Y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(Y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(Y_test, best_preds)))

# Evaluating